In [1]:
!pip install nltk


In [2]:
!pip install pandas

In [60]:
import pandas as pd
import nltk
from nltk.corpus import twitter_samples
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.classify import NaiveBayesClassifier
from sklearn.naive_bayes import MultinomialNB
import nltk.classify.util

nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


df=pd.read_csv('youtoxic_english_1000(2).csv',usecols=[2,15])
# podzial na texty i ich kasyfikacje
i=0
banedComments=[]
notBanedComments=[]
textsArray=df.drop("IsBaned",axis=1).to_numpy()
IsBanedArray=df["IsBaned"].to_numpy()

# funkcja do tworzenia arrayow z labelem true i zlabelem false
def add_to_array(text,index):
  # print(textsArray[index][0])
  if(text==True):
    banedComments.append(textsArray[index][0])
  else:
    notBanedComments.append(textsArray[index][0])
# tworzenie arrayow
for i in range(len(IsBanedArray)):
  add_to_array(IsBanedArray[i],i)

# robiwnie labelow i ich tasowanie
comments=banedComments+notBanedComments
labels = ['True'] * len(banedComments) + ['False'] * len(notBanedComments)
# comments
# Shuffle the dataset
combined = list(zip(comments, labels))
random.shuffle(combined)
comments, labels = zip(*combined)
# array to text i tokenizacja
sample_text = ','.join(comments)
tokens = word_tokenize(sample_text)

# usuwanie tokenów
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]
filtered_tokens = remove_stopwords(tokens)
# stematyzacja i lematyzacja tokenow
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

# stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
# lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]



# Convert comments to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X = vectorizer.fit_transform(comments).toarray()

# Create feature sets for training and testing

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

# Train a Naive Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Evaluate accuracy
accuracy = classifier.score(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# ////////////////////////////////////////////////////////////////////
# function to get precision recall and accuracy
def classiffy_train_set(sentence):

  classification = classifier.predict(sentence)
  return classification


# ocena modela
# predictions=[]
number=300
TP=0
TN=0
FP=0
FN=0
# for i in range(number):

predictions=classiffy_train_set(X_test)

for i in range(number):
  if(str(y_test[i])=="True"):
    if(str(predictions[i])==str(y_test[i])):
      TP=TP+1
    else:
      FN=FN+1
  else:
    if(str(predictions[i])==str(y_test[i])):
      TN=TN+1
    else:
      FP=FP+1
print ("true positive=  "+str(TP))
print ("true negative=  "+str(TN))
print ("false positive=  "+str(FP))
print ("false negative=  "+str(FN))
precision=TP/(TP+FP)
recall=TP/(TP+FN)
accuracy=(TP+TN)/number
print("precision= "+str(precision))
print("recall= "+str(recall))
print("accuracy= "+str(accuracy))

# definicja nowego textu(nie w training set)
def classiffy_sentence(text):

    # Preprocess the input text
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    # Convert tokens back to a single string
    preprocessed_text = " ".join(tokens)
    # Transform the text to TF-IDF vector
    text_vector = vectorizer.transform([preprocessed_text]).toarray()
    # Predict the label using the classifier
    prediction = classifier.predict(text_vector)

    return prediction[0]


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Accuracy: 74.00%
true positive=  89
true negative=  133
false positive=  33
false negative=  45
precision= 0.7295081967213115
recall= 0.664179104477612
accuracy= 0.74


In [61]:
print(classiffy_sentence("This comment seems offensive and should be banned"))


True
True
